In [1]:
from numba import cuda, float32
import numpy as np
import math
from time import time



In [2]:
# define variable for block size
# thread per block = BLOCK_SIZE x BLOCK_SIZE 
# It will determines: 1) CUDA execution configuration 2) size of shared memory in every SM
BLOCK_SIZE = 16


In [3]:
# define kernel function of matmul C = A * B without shared memory (for comparison)
@cuda.jit
def matmul(A, B, C):
    #  Here can also use “row, col = cuda.grid(2)” 
    row = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    col = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
    
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp



In [4]:
# define kernel function of matmul withshared memory 
@cuda.jit
def matmul_shared_memory(A, B, C):
    # Create shared memoy buffer for 2 matrixies in every SM by using, in same size as the CUDA block 
    # Note: we will see these 2 buffers in every SM’s shared memory
    # The shared-memory matrixes are defined at kernel function level, 
    # but they can be accessed by all the threads in the current SM
    sA = cuda.shared.array(shape=(BLOCK_SIZE, BLOCK_SIZE), dtype=float32)
    sB = cuda.shared.array(shape=(BLOCK_SIZE, BLOCK_SIZE), dtype=float32)

    # get the current’s thread’ index in the current block 
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    # get the current’s thread’ index in the grid (we can also use “row, col = cuda.grid(2)” here)
    row = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    col = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
    
    # 进行简单越界判断
    # 根据矩阵乘法，C的大小是A的行数(shape[0])、B的列数(shape[1])
    # 当(row, col)越界时退出
    if row >= C.shape[0] or col >= C.shape[1]:
        return

    # 计算按照当前tA、tB的大小, 需要拆分成多少个子block
    exec_num = int(math.ceil(1.0 * A.shape[1] / BLOCK_SIZE))

    tmp = 0.


    # 对于结果matrix C中的每一个thread，按照”蓝>橙>”顺序，逐个处理A和B中的子模块
    # 下面的注释以第一轮循环处理的蓝色block为例，解释内循环的处理流程
    for m in range(exec_num):
        # 每个thread根据其在目标C里的位置，将A和B蓝色block里对应位置上的那个数据， 
        # copy到shared memory里 sA和sB的对应位置上
        # 第一轮循环后，两个蓝色block里的数据会被对应copy到黄色clock所在SM的sA和sB里
        # 第二轮循环后，两个橙色block里的数据会同样被copy该SM的sA和sB里
        # 覆盖之前蓝色block的数据
        # 重复以上步骤，直至完成
        sA[tx, ty] = A[row, ty + m * BLOCK_SIZE]
        sB[tx, ty] = B[tx + m * BLOCK_SIZE, col]
        # 线程同步，等待Block中所有Thread预加载结束才执行下一步
        cuda.syncthreads()

        # 在每一轮循环里，都需要完成两个同色block的矩阵乘法，并累加到tmp里
        # 第一轮循环，用两个蓝色block里的数据求thread点的向量积，累加到tmp里
        # 第二轮循环，用两个橙色block里的数据求thread点的向量积，累加到tmp里
        # 循环完成后，temp中就保留了A x B = C在该当前thread点的值
        for n in range(BLOCK_SIZE):
            tmp += sA[tx, n] * sB[n, ty]

        # 线程同步，等待Block中所有Thread计算结束
        cuda.syncthreads()

    #循环完成后，temp中就保留了A x B = C在该当前thread点的值
    C[row, col] = tmp



In [5]:
# 初始化矩阵
M = 6000
N = 4800
P = 4000
A = np.random.random((M, N)) # 随机生成的 [M x N] 矩阵
B = np.random.random((N, P)) # 随机生成的 [N x P] 矩阵

A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
C_device = cuda.device_array((M, P)) # [M x P] 矩阵

# 执行配置
threads_per_block = (BLOCK_SIZE, BLOCK_SIZE)
blocks_per_grid_x = int(math.ceil(A.shape[0] / BLOCK_SIZE))
blocks_per_grid_y = int(math.ceil(B.shape[1] / BLOCK_SIZE))
blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

start = time()
matmul[blocks_per_grid, threads_per_block](A_device, B_device, C_device)
cuda.synchronize()
print("matmul time :" + str(time() - start))

start = time()
matmul_shared_memory[blocks_per_grid, threads_per_block](A_device, B_device, C_device)
cuda.synchronize()
print("matmul with shared memory time :" + str(time() - start))
C = C_device.copy_to_host()

# 验证正确性
if np.allclose(C, np.dot(A, B)):
    print("matmul with shared memory, result correct")


matmul time :4.7024455070495605
matmul with shared memory time :2.4234275817871094
matmul with shared memory, result correct
